In [1]:
import matplotlib.pyplot as plt
import forex as fx
import json
import numpy as np
import gymnasium as gym
import ray
from ray.rllib.algorithms.ppo import PPOConfig
import random
from ray.tune.registry import register_env
import os
from tqdm import tqdm
t = fx.hr("EUR_USD")







YF.download() has changed argument auto_adjust default to True


In [55]:

ENV_CONFIG = {"pair" : "EUR_USD",
              "time" : "hour"}

class sol4_env(gym.Env):
    def __init__(self, config=None):


        # self.history = fx.min("EUR_USD").to_numpy()[0][:,1:5]
        # if "pair" not in config.keys():
        if config["time"] in ["day", "d", "Day"]:
            self.history = fx.day(config["pair"]).to_numpy()[0][:,1:5]
        elif config["time"] in ["minute", "min", "m"]:
            self.history = fx.min(config["pair"]).to_numpy()[0][:,1:5]
        else:
            self.history = fx.hr(config["pair"]).to_numpy()[0][:,1:5]
        
        self.hr = fx.hr(config["pair"]).to_numpy()[0][:,1:5]
        self.min = fx.min(config["pair"]).to_numpy()[0][:,1:5]
        self.day = fx.day(config["pair"]).to_numpy()[0][:,1:5]
        self.history = self.hr
                    
        arr = self.history
        a,b,c = fx.algo.bollinger_bands(arr)
        d,e,f = fx.algo.macd(arr)
        g = fx.algo.rsi(arr)
        h, _i = fx.algo.smi(arr)


        hstackarr = [arr, a,b,c,d,e,f,g,h,_i]
        mval = min([i.shape for i in hstackarr])[0]


        for i in range(len(hstackarr)):
            hstackarr[i] = hstackarr[i][hstackarr[i].shape[0] - mval : ]
            if i != 0:
                hstackarr[i] = hstackarr[i].reshape(len(hstackarr[i]),1)

        self.history = np.hstack(hstackarr)

            
        self.action_space = gym.spaces.Discrete(3) 
        r1 = random.randint(0,len(self.history) - 128)
        _state = self.history[r1: r1+64]
        self._s1 = _state[:32]#.flatten()
        self._s2 = _state[32:]#.flatten()
        print(f"SHAPE #######################\n{self._s1.shape}")
        self.c = 0
        self.iters = 50000
        self.total_reward = 0
        self.observation_space = gym.spaces.Box(0.0, 2.0, shape=(32,4), dtype=np.float64)
        

        ### test
        self.monie = 100
        

    def reset(self, seed=None, options=None):

        r1 = random.randint(0,len(self.history) - 128)
        _state = self.history[r1: r1+64]
        self._s1 = _state[:32]#.flatten()
        self._s2 = _state[32:]#.flatten()
        # print(f"SHAPE #########################\n{self._s1.shape} {self._s2.shape}")
        self.counting = 0
        self.monie=100

        # return np.array([1.0]), {}
        return self._s1, {"total_reward" : self.total_reward}

    def step(self, action):
        
        reward = 0
        terminated = False
        
        s1_v = self._s1[-1][0]
        s2_v = self._s2[-1][0]
        # print(self._s1[-1])

        if action == 0:
            # buy 
            # print("[BUY]")
            if s1_v < s2_v:
                reward = s2_v - s1_v
                reward = 1
            else:
                reward = -(s2_v - s1_v)
                reward = -1  
        if action == 1:
            # sell
            # print("[SELL]")
            if s1_v > s2_v:
                reward = s2_v - s1_v
                reward = 1
            else:
                reward = -(s2_v - s1_v)
                reward = -1
        
        if action == 2:
            # hold
            # print("[HOLD]")
            reward = 0
            
        
        r1 = random.randint(0,len(self.history) - 128)
        _state = self.history[r1: r1+64]
        self._s1 = _state[:32]#.flatten()
        self._s2 = _state[32:]#.flatten()

        # hist_stats = {
        # "total_reward": self.total_reward,
        # "average_reward": self.total_reward / (self.counting if self.counting > 0 else 1),
        # "episode_length": self.counting
        # }

        self.monie += reward
        # print(self.monie)
        if reward < -0.5:
            return self._s1, reward, True, False, {}#{"total_reward": reward, "hist_stats": hist_stats}

        self.counting += 1
        self.total_reward += reward
        
        # Return next observation, reward, terminated, truncated, and info dict (now with hist_stats)
        return self._s1, reward, False, False, {}#{"custom_metrics" : "Hi"}#{"total_reward": reward, "hist_stats": hist_stats}

t = sol4_env({"pair" : "EUR_USD",
              "time" : "hr"})


t.history.max()


SHAPE #######################
(32, 13)


1.1277772188186646

In [ ]:
arr = fx.hr("EUR_USD").chlo
a,b,c = fx.algo.bollinger_bands(arr)
d,e,f = fx.algo.macd(arr)
g = fx.algo.rsi(arr)
h, _i = fx.algo.smi(arr)


hstackarr = [arr, a,b,c,d,e,f,g,h,_i]
mval = min([i.shape for i in hstackarr])[0]


for i in range(len(hstackarr)):
    hstackarr[i] = hstackarr[i][hstackarr[i].shape[0] - mval : ]
    if i != 0:
        hstackarr[i] = hstackarr[i].reshape(len(hstackarr[i]),1)

np.hstack(hstackarr)


array([[ 1.09063148,  1.09146476,  1.08932459, ...,  0.48761028,
        -0.40110843, -0.09505875],
       [ 1.09721303,  1.09745395,  1.09158385, ...,  0.65952774,
         0.85071797,  0.37782961],
       [ 1.09553027,  1.09721303,  1.0952903 , ...,  0.62992135,
         0.9309569 ,  0.65439326],
       ...,
       [ 1.0789814 ,  1.07968044,  1.07886505, ...,  0.35643582,
        -0.99285585, -0.97660369],
       [ 1.07921433,  1.0793308 ,  1.07886505, ...,  0.42160073,
        -0.99607496, -0.98633932],
       [ 1.07979703,  1.07979703,  1.07921433, ...,  0.44337695,
        -0.99698666, -0.99166299]])

In [15]:
arr

array([[1.08061373, 1.08283699, 1.08049703, 1.08283699],
       [1.08178282, 1.08178282, 1.08038032, 1.08061373],
       [1.08119798, 1.08189976, 1.08073056, 1.08154881],
       ...,
       [1.0789814 , 1.07968044, 1.07886505, 1.07956386],
       [1.07921433, 1.0793308 , 1.07886505, 1.0789814 ],
       [1.0793308 , 1.07944727, 1.07921433, 1.07921433]])